In [24]:
import os
import pandas as pd
import cufflinks as cf
from storage import FStruct, FStorage, SECURITIES, DIVIDENDS, TRADE_HISTORY, MARKETDATA, DIVIDENDS_PROCESSED

cf.set_config_file(offline=True)
cf.go_offline()
root_dir = os.path.abspath("")
fstruct = FStruct(root_dir)
fstorage = FStorage(root_dir)

In [25]:
sec_id = "SBER"
sale_year = 2020
sale_month = 3
broker_commission = 0.07*0.01
tax_value = 0.13

sale_month_start = pd.Timestamp(sale_year, sale_month, 1)
sale_month_finish = pd.Timestamp(sale_year, sale_month, sale_month_start.days_in_month)

hist = fstorage.open_data(TRADE_HISTORY, sec_id, index_col="TRADEDATE").sort_values("TRADEDATE", ascending = True)[["LEGALCLOSEPRICE"]]
hist_sale_month = hist.loc[(hist.index >= sale_month_start) & (hist.index <= sale_month_finish), "LEGALCLOSEPRICE"]
last_div_date = pd.Timestamp(hist_sale_month.index.values[hist_sale_month.size // 2])
# sale_price = mean by sale month
sale_price = hist_sale_month.mean()
show_date = sale_month_start + pd.DateOffset(days=-30)

divs = fstorage.open_data(DIVIDENDS_PROCESSED, sec_id, index_col="t2date").sort_values("t2date", ascending = True)[["value"]]
divs = divs.loc[divs.index<=last_div_date]
div_sum = divs["value"].sum()

In [29]:
res = hist.join(divs)
res["div_cum_sum"] = res["value"].fillna(0).cumsum()
res["inv_div_cum_sum"] = div_sum - res["div_cum_sum"]

res["commission"] = (sale_price + res["LEGALCLOSEPRICE"])*broker_commission

res["income_tax_value"] = 0.0
res.loc[last_div_date - pd.DateOffset(years=3) <= res.index, "income_tax_value"] = tax_value
res["tax"] = (sale_price - res["LEGALCLOSEPRICE"] - res["commission"])*res["income_tax_value"] + res["inv_div_cum_sum"] * tax_value

res["income"] = (res["inv_div_cum_sum"] + sale_price - res["LEGALCLOSEPRICE"])
res["clear_income"] = res["income"] - res["tax"] - res["commission"]

res = res.loc[res.index<last_div_date]
res["year_percent"] = res["income"]*100.0*365.25/(res["LEGALCLOSEPRICE"]*(last_div_date - res.index).days)
res["clear_year_percent"] = res["clear_income"]*100.0*365.25/(res["LEGALCLOSEPRICE"]*(last_div_date - res.index).days)

# res[res.index<show_date].iplot(y=["year_percent", "clear_year_percent"], fill=True, xTitle="Dates", yTitle="Percents", title="Interest income per annum ({})".format(sec_id), dimensions=(1400, 800))